In [6]:
import yfinance as yf
import pandas as pd
from pathlib import Path
import math 

In [7]:
result_folder = Path.cwd().parent / 'data'

symbols = pd.read_csv(result_folder / 'simple_screener_results.csv')['Ticker'].tolist()
symbols = pd.Series(symbols).unique()
backup = pd.read_csv(result_folder / 'simple_screener_results.csv')['Ticker']
if len(backup) > 0:
    backup.to_csv(result_folder / 'known_tickers.csv', index=False)
owned_tickers = pd.read_csv(result_folder / 'owned_tickers.csv')['Ticker'].tolist()

In [8]:
df = pd.DataFrame(columns=[
    'Ticker', 
    'short_name', 
    'recommendation_score',
    'value_score',  
    'analyst_mean%', 
    'insider_buy%', 
    'sector', 
    'industry', 
    'country',
    'P/E (TTM)', 
    'ROA%',
])

for symbol in symbols: 
    ticker = yf.Ticker(symbol)
    try:
        analyst_mean = 0
        if 'mean' in ticker.analyst_price_targets and ticker.info['currentPrice']:
            analyst_mean = ticker.analyst_price_targets['mean'] / ticker.info['currentPrice'] - 1
        
        roa = ticker.info['returnOnAssets']*100
        pe = ticker.info['trailingPE']
        
        value_score = format(2.6 + math.log(roa+5) - math.log(pe+25), '.2f')
        
        insider_buy = 0
        if 'Shares' in ticker.insider_purchases.columns:
            insider_buy = format(ticker.insider_purchases.loc[ticker.insider_purchases.index[4], 'Shares'], '.2f')
            if insider_buy == '<NA>':
                insider_buy = 0

        recommendation_score = format(float(value_score) + analyst_mean * 1.25 + float(insider_buy) * 2.0, '.2f')

        df.loc[symbol] = [
            symbol, 
            ticker.info['shortName'], 
            recommendation_score,
            value_score,
            format(analyst_mean * 100, '.2f'), 
            insider_buy,
            ticker.info['sector'], 
            ticker.info['industry'], 
            ticker.info['country'],
            format(pe, '.2f') if isinstance(pe, (int, float)) else 'N/A', 
            format(roa, '.2f') if isinstance(roa, (int, float)) else 'N/A',
        ]
    except Exception as e:
        print(f"Error with {symbol}: {e}. Removed Ticker from list.")

df.to_csv(result_folder / 'simple_screener_results.csv', index=False)
display(df)

print(f'Got data for {len(df)} stocks')

,Ticker,short_name,recommendation_score,value_score,analyst_mean%,insider_buy%,sector,industry,country,P/E (TTM),ROA%
MATAS.CO,MATAS.CO,Matas A/S,1.10,0.88,17.96,0,Consumer Cyclical,Specialty Retail,Denmark,24.18,3.84
TRIFOR.CO,TRIFOR.CO,Trifork Group AG,1.21,0.86,27.64,0,Technology,Information Technology Services,Switzerland,14.42,1.92
QQ.L,QQ.L,QINETIQ GROUP PLC ORD 1P,1.44,1.31,10.19,0,Industrials,Aerospace & Defense,United Kingdom,18.00,6.89
RNMBY,RNMBY,Rheinmetall AG,0.34,0.34,0.00,0,Industrials,Aerospace & Defense,Germany,90.20,7.06
SAABF,SAABF,Saab AB,0.43,0.43,0.00,0,Industrials,Aerospace & Defense,Sweden,52.25,3.80
BCKIY,BCKIY,Babcock International Group Plc,1.03,1.03,0.00,0,Industrials,Engineering & Construction,United Kingdom,23.58,5.08
BAESY,BAESY,BAE Systems PLC,0.67,0.88,-16.85,0,Industrials,Aerospace & Defense,United Kingdom,27.68,4.40
IVSO.ST,IVSO.ST,INVISIO AB,1.38,1.39,-0.68,0,Technology,Communication Equipment,Sweden,56.85,19.45
NSKFF,NSKFF,Kongsberg Gruppen ASA,0.87,0.87,0.00,0,Industrials,Aerospace & Defense,Norway,47.57,7.82
GMAB,GMAB,Genmab A/S,2.86,2.25,49.17,0,Healthcare,Biotechnology,Denmark,11.56,20.66


Got data for 45 stocks


### Single import

In [9]:
ticker = "UBER"
print(f"P/E: {yf.Ticker(ticker).info['trailingPE']:.2f}")
print(f"ROA%: {yf.Ticker(ticker).info['returnOnAssets']*100:.2f}")

P/E: 15.79
ROA%: 5.21
